In [ ]:
import psycopg2
import random
from datetime import datetime, time
from time import sleep

# Sample product data
PRODUCTS = [
    "Banana", "Apple", "Orange", "Mango", "Grapes", "Blueberry", "Strawberry"
]
CATEGORIES = ["Fruits", "Vegetables", "Dairy", "Meat", "Bakery"]

# Establish the connection
conn = psycopg2.connect(
    dbname='tutorialdb',
    user='postgres',
    password='P0stgres@2020',
    host='localhost',
    port='5432'
)

# Create a cursor object
cur = conn.cursor()
# Product and Category mapping
products_categories = {
    'Fruits': ['Banana', 'Apple'],
    'Bakery': ['Bread'],
    'Diary': ['Butter','Eggs','Milk'],
    'Meats': ['Chicken'],
    'Vegetables': ['Carrot', 'Tomato', 'Potato']
}

while True:
    # Generate random data
    category = random.choice(list(products_categories.keys()))
    product_name = random.choice(products_categories[category])
    quantity = random.randint(1, 10)
    
    # Lookup price for the given product
    cur.execute("SELECT Price FROM sample WHERE ProductName = %s LIMIT 1", [product_name])
    result = cur.fetchone()
    if result:
        price = result[0]
    else:
        price = 1.0  # Default price if the product doesn't exist. Modify as needed.

    # Get the next TransactionID
    cur.execute("SELECT MAX(TransactionID) FROM sample")
    last_id = cur.fetchone()[0]
    if last_id is None:
        last_id = 0
    next_id = last_id + 1
    
    # Insert data into the table
    current_date = datetime.now().date()
    current_time = datetime.now().time()
    cur.execute("INSERT INTO sample (TransactionID, Date, Time, ProductName, Category, Price, Quantity) VALUES (%s, %s, %s, %s, %s, %s, %s)",
                [next_id, current_date, current_time, product_name, category, price, quantity])
    
    # Commit the transaction
    conn.commit()

    # Wait before the next insert
    sleep_time = random.randint(3, 1000)
    time.sleep(sleep_time)


cur.close()
conn.close()